## Deliverable 2. Create a Customer Travel Destinations Map.

In [30]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [7]:
# 1. Import the WeatherPy_database.csv file. 
# Store the CSV you  created in Deliverable one into a DataFrame.
weather_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
weather_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description
0,0,Rikitea,-23.1203,-134.9692,78.13,72,75,7.49,PF,broken clouds
1,1,Port Alfred,-33.5906,26.8910,65.91,55,96,22.21,ZA,overcast clouds
2,2,Chokurdakh,70.6333,147.9167,-10.95,97,60,2.06,RU,broken clouds
3,3,Punta Arenas,-53.1500,-70.9167,51.80,71,40,9.22,CL,scattered clouds
4,4,Meulaboh,4.1363,96.1285,77.81,83,100,1.81,ID,overcast clouds


In [8]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [16]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df1 = weather_data_df.loc[(weather_data_df["Max Temp"] <= max_temp) & \
                                       (weather_data_df["Max Temp"] >= min_temp)]
preferred_cities_df = preferred_cities_df1.dropna()
preferred_cities_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description
0,0,Rikitea,-23.1203,-134.9692,78.13,72,75,7.49,PF,broken clouds
4,4,Meulaboh,4.1363,96.1285,77.81,83,100,1.81,ID,overcast clouds
5,5,Puerto Ayora,-0.7393,-90.3518,79.00,89,75,9.22,EC,light rain
8,8,Kapaa,22.0752,-159.3190,77.00,73,90,9.22,US,light rain
12,12,Gambela,8.2500,34.5833,79.99,49,38,2.17,ET,scattered clouds


In [18]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Country                0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
#Redundant- this step was completed above with .dropna() - I left 4a in as a verification

In [19]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Rikitea,PF,78.13,broken clouds,-23.1203,-134.9692,
4,Meulaboh,ID,77.81,overcast clouds,4.1363,96.1285,
5,Puerto Ayora,EC,79.00,light rain,-0.7393,-90.3518,
8,Kapaa,US,77.00,light rain,22.0752,-159.3190,
12,Gambela,ET,79.99,scattered clouds,8.2500,34.5833,
17,Georgetown,MY,78.80,scattered clouds,5.4112,100.3354,
22,Tutoia,BR,77.86,overcast clouds,-2.7619,-42.2744,
26,Atuona,PF,80.80,few clouds,-9.8000,-139.0333,
28,Caravelas,BR,75.60,overcast clouds,-17.7125,-39.2481,
31,Mahebourg,MU,77.00,light rain,-20.4081,57.7000,


In [20]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [24]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df[hotel_df["Hotel Name"]!= '']
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Rikitea,PF,78.13,broken clouds,-23.1203,-134.9692,Pension Maro'i
4,Meulaboh,ID,77.81,overcast clouds,4.1363,96.1285,Kost Pria
5,Puerto Ayora,EC,79.00,light rain,-0.7393,-90.3518,Finch Bay Galapagos Hotel
8,Kapaa,US,77.00,light rain,22.0752,-159.3190,Sheraton Kauai Resort at Coconut Beach
12,Gambela,ET,79.99,scattered clouds,8.2500,34.5833,Solomon Debebe Hotel
17,Georgetown,MY,78.80,scattered clouds,5.4112,100.3354,Cititel Penang
22,Tutoia,BR,77.86,overcast clouds,-2.7619,-42.2744,Pousada Guará's
26,Atuona,PF,80.80,few clouds,-9.8000,-139.0333,Villa Enata
28,Caravelas,BR,75.60,overcast clouds,-17.7125,-39.2481,Pousada dos Navegantes
31,Mahebourg,MU,77.00,light rain,-20.4081,57.7000,Shandrani Beachcomber Resort & Spa


In [27]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [28]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and </dd><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [32]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))